## Capstone Notebook 1: Data Cleaning

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm

In [50]:
Bixi_2021_dataset

,start_date,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member
0,2021-06-29 17:46:28.653,10,2021-06-29 19:33:25.700,10,6417,0
1,2021-04-25 18:02:39.110,10,2021-04-25 18:13:26.139,188,647,0
2,2021-08-08 18:50:31.633,10,2021-08-08 18:58:57.585,778,505,1
3,2021-10-11 17:50:15.346,10,2021-10-11 18:25:31.726,504,2116,1
4,2021-09-10 15:51:58.050,10,2021-09-10 16:21:01.403,780,1743,1
...,...,...,...,...,...,...
5566346,2021-11-10 18:17:49.654,1140,2021-11-10 18:24:40.643,928,410,1
5566347,2021-11-11 14:55:04.039,1140,2021-11-11 14:59:41.446,1140,277,0
5566348,2021-11-12 14:50:15.504,1140,2021-11-12 15:05:37.379,1140,921,1
5566349,2021-11-13 07:00:16.679,1140,2021-11-13 07:15:10.760,945,894,1


In [57]:
pop_dense

,geom,densite_population_2021,population_2021
0,MULTIPOLYGON (((-73.56669371689269 45.46011224...,14536.7,455
1,MULTIPOLYGON (((-73.60259070750847 45.53181946...,13545.5,596
2,MULTIPOLYGON (((-73.66866107016182 45.54963768...,19799,394
3,MULTIPOLYGON (((-73.60334317309518 45.53099231...,17777.8,640
4,MULTIPOLYGON (((-73.75639055608751 45.44768164...,11021.5,410
...,...,...,...
3913,MULTIPOLYGON (((-73.5556563788921 45.496278163...,14436.7,2281
3914,MULTIPOLYGON (((-73.66851264217964 45.51959762...,3093.4,1474
3915,MULTIPOLYGON (((-73.62337325854918 45.46340923...,36104.4,899
3916,MULTIPOLYGON (((-73.61656130281877 45.59187990...,4204.7,1105


### Bixi Trips

| Column | Description |
|---|---|
| start_date | Date and Time of when trip started |
| emplacement_pk_start | Station code from station where the recorded trip began |
| end_date | Date and Time of when trip ended |
| emplacement_pk_end | Station code from station where the recorded trip ended |
| duration_sec | length of time of recorded trip |
| is_member | Binary column of whether the cyclist was a Bixi member of not |

### Bixi stations

| Column | Description |
|---|---|
| pk | Station code |
| name | Name of station |
| longitude | Longitudinal location of station |
| latitude | Latitudinal location of station |

### Points of Interest

| Column | Description |
|---|---|
| ID | ID column for each point of interest |
| Famille | A low granularity breakdown of different types of points of interest |
| Catégorie | A somewhat high granularity breakdown of different types of points of interest |
| Nom français | The name of points of interest |
| Nom court | A shortened name for each point of interest |
| Type | A very high granularity breakdown of different types of points of interest |
| Numéro | Civic number |
| rue | Street name |
| Étage | Number of floors for each Point of interest |
| Bureau | Desk space for Commerical Points of interest |
| Ville | City the Point of Interest is in (Montreal for all of them) |
| Code Postal | Post code of the Points of interest |
| Arrondissement | The borough of Montreal |
| classification | The building classification (construction laws) |
| longitude | longitudinal location |
| latitude | latitudinal location |

### Population Density

| Column | Description |
|---|---|
| adidu | ???? |
| geom | A column of multipolygons, which make up the city of Montreal |
| densite_population_2021 | The population density of each multipolygon | 
| population_2021 | The population of each multipolygon |
| Unnamed 4-876 | specific areas within various multipolygons, impossible to cleanly map from a csv |

For this capstone, I am building a model to estimate the potential success of a new bixi station in Montreal. An additional goal for this project is for the model I create to be adjustable to other cities with or without bike hire schemes. The key data sources I am using for this project are Bixi data from 2021 (trip data and station data), Points of Interest data from 2022 (based on a pedestrianisation project in Montreal, and included to help identify what surrounds bixi stations) and Population Density data from 2021 (Bixi claims to use population density data when deciding were to place stations).

This notebook, will contain basic descriptions of these datasets as well as some data cleaning (most these datasets have duplicate rows and Null value columns). The advanced EDA and wireframe/feature creation will be covered in the next notebook, as the 4 datasets (the Bixi dataset is split in half) need to be combined into 1 dataset that can be modelled.

In [23]:
# Importing the two open source Bixi datasets
Bixi_2021_dataset = pd.read_csv("C:/Users/ckedd/Documents/Capstone data/2021_donnees_ouvertes.csv")
Bixi_2021_stations = pd.read_csv("C:/Users/ckedd/Documents/Capstone data/2021_stations.csv")

In [53]:
# Importing the open source POI dataset
Points_of_interest = pd.read_csv("C:/Users/ckedd/OneDrive/Documents/Brain station/Capstone dataset/points of interest/lieux_d_interet.csv")

In [25]:
# Importing my existing EDA file with pre-set EDA functions within
import ds_utils as EDA

## Bixi 2021 Trips Dataset

Breaking down the basics of this dataset and checking for duplicates and Null values

In [26]:
Bixi_2021_dataset.head()

,start_date,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member
0,2021-06-29 17:46:28.653,10,2021-06-29 19:33:25.700,10,6417,0
1,2021-04-25 18:02:39.110,10,2021-04-25 18:13:26.139,188,647,0
2,2021-08-08 18:50:31.633,10,2021-08-08 18:58:57.585,778,505,1
3,2021-10-11 17:50:15.346,10,2021-10-11 18:25:31.726,504,2116,1
4,2021-09-10 15:51:58.050,10,2021-09-10 16:21:01.403,780,1743,1


In [27]:
# Creating a duplicate dataset, before running an EDA to avoid affecting the main dataset
df_bixi = Bixi_2021_dataset.copy(deep = True)
df_bixi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566351 entries, 0 to 5566350
Data columns (total 6 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   start_date            object
 1   emplacement_pk_start  int64 
 2   end_date              object
 3   emplacement_pk_end    int64 
 4   duration_sec          int64 
 5   is_member             int64 
dtypes: int64(4), object(2)
memory usage: 254.8+ MB


In [28]:
# Running an EDA test, which looks at rows, duplicates, size and Null values in a DataFrame
EDA.basic_eda(df_bixi, 'df_bixi')

DF_BIXI

Rows: 5566351 	 Columns: 6

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 6
Percentage dupe rows: 0.0%

start_date              object
emplacement_pk_start     int64
end_date                object
emplacement_pk_end       int64
duration_sec             int64
is_member                int64
dtype: object
-----



From the EDA function, I can see that there are 6 duplicate rows. This could be a mistake in recording or two of people cycling at the exact same time. As one of the columns is recording how long the trip is in seconds, I find it unlikely that it is two separate people cycling together, especially since if two people cycling together caused duplicates, there would be far more duplicates recorde. As a result, and due to tiny number of duplicates, I will simply drop the duplicate rows, with keep= 'first' so that the first of the duplicates isn't deleted.

In [29]:

df_bixi.drop_duplicates(keep='first', inplace=True)

df_bixi.duplicated().sum()

0

As duplicate values were the only issues with the dataset, the data cleaning is done so I can export the DataFrame as a csv file to re-import for my EDA notebook.

In [30]:
from importlib.resources import path
from pathlib import Path  
filepath = Path('C:/Users/ckedd/Documents/Capstone data/Bixi_trips.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
# I am exporting the modified copied DataFrame, not the original
df_bixi.to_csv(filepath)  

## Points of interest Data cleaning


In [31]:
Points_of_interest.head()

,ID,Famille,Catégorie,Nom français,Nom court,Type,Numéro,rue,Étage,Bureau,Ville,Code postal,Arrondissement,Classification,Longitude,Latitude
0,1,Culturel,Établissement culturel,Maison du Meunier,NaN,Musée et centre d'interprétation / d'exposition,10897,Rue du Pont,0,,Montréal,H2B 1Z4,Ahuntsic-Cartierville,niveau 3,-73.661102,45.574862
1,2,Culturel,Établissement culturel,Maison du Pressoir,NaN,Musée et centre d'interprétation / d'exposition,10865,Rue du Pressoir,0,,Montréal,H2B 2L1,Ahuntsic-Cartierville,niveau 3,-73.659553,45.575970
2,3,Commercial,Regroupement de commerces,Marché Central,NaN,Centre commercial,9187,Boulevard de l'Acadie,0,,Montréal,H4N 3K1,Ahuntsic-Cartierville,niveau 1,-73.654956,45.534023
3,4,Culturel,Établissement culturel,Musée des Sœurs de la Providence,NaN,Musée et centre d'interprétation / d'exposition,12055,Rue Grenet,0,,Montréal,H4J 2J5,Ahuntsic-Cartierville,niveau 3,-73.713635,45.529374
4,5,Récréatif / sportif,Parc et autre espace vert,Parc Ahuntsic,NaN,Parc,,,0,,Montréal,H3L 2E5,Ahuntsic-Cartierville,niveau 2,-73.664710,45.555801


The Points of interest Dataset, is required for this capstone, as a way to determine what surrounds the stations (popular stations may have similar points of interest surrounding them). There are 3 different columns in the Points of Interest dataset, that can be used to classify the different points. For the purposes of this capstone, I will only keep the "Type" and "Famille" classification columns. The "Name"(but in French) column will be kept incase I need to look up each point, but other columns seem to either describe the points of interest (which isn't helpful without data on the number of people who use them), or further breaks down the classification for the points of interest, to such a low granularity that they cannot be used for trends (Famille is extremely broad for categorising and Type allows for spcific data types to be categorised as well e.g. Metro stations).

As a result I will drop columns aside from the location data (lon and lat), Name data, Type and Famille data.

In [32]:
Points_of_interest['Type'].value_counts()

Parc                                                      865
École primaire                                            270
Organisme communautaire et centre de loisirs              261
École secondaire et professionnelle                       177
Hôtel                                                     122
Art public                                                108
Poste d'attente de taxis                                   97
Jardin communautaire                                       90
Bâtiment historique / patrimonial                          82
Théâtre / salle de spectacle / petit lieu de diffusion     77
Station de métro                                           64
Centre sportif intérieur                                   61
Caserne de pompiers                                        55
Cégep et université                                        53
Rue piétonne                                               52
Bibliothèque publique                                      49
Place pu

In [33]:
Points_of_interest['Famille'].value_counts()

Récréatif / sportif    1390
Service public         1041
Culturel                557
Commercial              243
Name: Famille, dtype: int64

In [34]:
EDA.basic_eda(Points_of_interest, 'Points_of_interest')

POINTS_OF_INTEREST

Rows: 3231 	 Columns: 16

Total null rows: 2739
Percentage null rows: 84.77%

Total duplicate rows: 0
Percentage dupe rows: 0.0%

ID                  int64
Famille            object
Catégorie          object
Nom français       object
Nom court          object
Type               object
Numéro             object
rue                object
Étage               int64
Bureau             object
Ville              object
Code postal        object
Arrondissement     object
Classification     object
Longitude         float64
Latitude          float64
dtype: object
-----



In [35]:
Points_of_interest.drop(columns=Points_of_interest[['Bureau','ID','Numéro', 'Nom court', 'Catégorie', 'rue', 'Étage', 'Ville', 'Code postal', 'Arrondissement', 'Classification']], inplace=True)

As only necessary columns are left in the Points_of_interest DataFrame, the last thing to clean, is the name of the location data (Longitude and Latitude), the Bixi dataset uses lower case ls, so for simplicity, I will change the Points_of_interest column names to match, this will also simplify function definitions in the EDA notebook.

In [36]:
Points_of_interest['latitude'] = Points_of_interest['Latitude']
Points_of_interest['longitude'] = Points_of_interest['Longitude']
Points_of_interest.drop(Points_of_interest[['Longitude', 'Latitude']], axis=1, inplace=True)

In [37]:
from importlib.resources import path
from pathlib import Path  
filepath = Path('C:/Users/ckedd/Documents/Capstone data/Points_of_interest.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
Points_of_interest.to_csv(filepath)  

## Bixi stations Data cleaning

In [38]:
Bixi_2021_stations.head()

,pk,name,latitude,longitude
0,10,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630
1,13,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959
2,17,Tolhurst / Fleury,45.544149,-73.667520
3,21,St-Christophe / Cherrier,45.520290,-73.568119
4,31,Lespérance / de Rouen,45.538839,-73.552779


In [39]:
bix_sta = Bixi_2021_stations

EDA.basic_eda(bix_sta, 'stations')

STATIONS

Rows: 829 	 Columns: 4

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 0
Percentage dupe rows: 0.0%

pk             int64
name          object
latitude     float64
longitude    float64
dtype: object
-----



The Station DataFrame will be the base of my data wireframe, due to each row recording one station. This means that there will be no granularity issues for the modelling process. As there are no issues I can just import the original file into my next notebook, however there still could be other issues with the dataset that I can't detect in basic data cleaning that only EDA can really fix.

## Population Density Data Cleaning

This data was provided by anagraph.io, and shows the population density in different polygons within Montreal. The goal for this data is to find out which polygon each station is in, then assigning a population density to them. To do this, I will first clean the data, before running advanced EDA in the next notebook. The only required data from this dataset is the Geom (which contains the polygon data) and the population columns (1 for population and 1 for population density).

In [41]:
pop_dense = pd.read_csv("C:/Users/ckedd/Documents/Brain_station/Capstone dataset/Population Density/demo2021_pop_densite_mtl.csv")

C:\Users\ckedd\AppData\Local\Temp\ipykernel_31468\1520332155.py:1: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,25

In [42]:
pop_dense.head()

,adidu,population_2021,densite_population_2021,geom,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 867,Unnamed: 868,Unnamed: 869,Unnamed: 870,Unnamed: 871,Unnamed: 872,Unnamed: 873,Unnamed: 874,Unnamed: 875,Unnamed: 876
0,24661221.0,455,14536.7,MULTIPOLYGON (((-73.56669371689269 45.46011224...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24661456.0,596,13545.5,MULTIPOLYGON (((-73.60259070750847 45.53181946...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24662533.0,394,19799,MULTIPOLYGON (((-73.66866107016182 45.54963768...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24661457.0,640,17777.8,MULTIPOLYGON (((-73.60334317309518 45.53099231...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24662253.0,410,11021.5,MULTIPOLYGON (((-73.75639055608751 45.44768164...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
pop_dense = pop_dense[['geom', 'densite_population_2021', 'population_2021']]

In [45]:
EDA.basic_eda(pop_dense, 'pop_dense')

POP_DENSE

Rows: 3918 	 Columns: 3

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 1252
Percentage dupe rows: 31.96%

geom                       object
densite_population_2021    object
population_2021            object
dtype: object
-----



As this data is based on geographical polygons, I will drop the duplicated rows, as they would be overlayed and therefore be usless, if kept in. I will still use the keep = 'first' to make sure that no data is lost, as losing a polygon would be disasterous for my EDA analysis. Then I will export the data, so that it can be imported into the next notebook.

In [47]:
pop_dense.drop_duplicates(keep='first', inplace = True)

In [48]:
from importlib.resources import path
from pathlib import Path  
filepath = Path('C:/Users/ckedd/Documents/Capstone data/pop_dense.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
pop_dense.to_csv(filepath)  